In [ ]:
import cv2
import math
from math import cos,sin
import numpy as np
from google.colab.patches import cv2_imshow

h = 0.7
alpha = math.radians(45)
f_l = 26
x0 = 0.072
y0 = 0.96
'''
h11 = f_l
h12 = x0*cos(alpha)
h13 = x0*h*cos(alpha)
h21 = 0
h22 = f_l*sin(alpha)+y0*cos(alpha)
h23 = y0*h*cos(alpha)-f_l*h*sin(alpha)
h31 = 0
h32 = cos(alpha)
h33 = h*cos(alpha)'''

h11 = 1
h12 = 0
h13 = 0
h21 = 0
h22 = sin(alpha)
h23 = -h*sin(alpha)
h31 = 0
h32 = cos(alpha)
h33 = h*cos(alpha)

pts_src = np.array([[296,0],[753,0],[1160,1920],[19,1920]])

pts_dst = np.array([[296,0],[753,0],[950,1920],[490,1920]])

h_m, status = cv2.findHomography(pts_src, pts_dst)

inv_h_matrix = np.array([[h11,h12,h13],[h21,h22,h23],[h31,h32,h33]])
print(inv_h_matrix)
h_matrix = np.linalg.inv(inv_h_matrix)
print(h_matrix)
mat_mul = np.matmul(inv_h_matrix,h_matrix)
print(mat_mul)

img = cv2.imread('/content/drive/MyDrive/GarbageClassifier/object2transform_homograpfy.jpg')
cv2_imshow(img)

im_out = cv2.warpPerspective(img, h_m, (img.shape[1],1300))
cv2_imshow(im_out)

In [ ]:
math.sqrt(535**2 + 1653**2)

1737.4216529098514

In [ ]:
import cv2
import math
from math import cos,sin,tan,sqrt
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# the class warps input frame perspective to overhead view and calculates relative to robot coordinates of the given within frame objects
class camera():
  def __init__(self,hor_fov,ver_fov,height = None,tilt_a = None):
    self.hor_fov = hor_fov
    self.ver_fov = ver_fov
    self.h = height
    self.tilt_a = tilt_a
    self.d_f =None
    self.d_r =None
    self.w_f =None
    self.w_r =None
    self.area_of_view_size = [None,None]
    self.area_of_view_pxl_size = [None,None]
    self.aov_object_coords = None # relative to robot

  def rectify_perspective(self,img,h = None,tilt_a = None,detections = []):
    if h is not None:
      self.h = h
    if tilt_a is not None:
      self.tilt_a = tilt_a
    if img is None:
      print('Given images is None')
      return
    self.d_f = self.h*tan(self.tilt_a - self.ver_fov)
    self.d_r = self.h*tan(self.tilt_a + self.ver_fov)
    self.area_of_view_size[1] = self.d_r - self.d_f

    self.w_f = sqrt(self.d_f**2+self.h**2)*tan(self.hor_fov)
    self.w_r = sqrt(self.d_r**2+self.h**2)*tan(self.hor_fov)
    self.area_of_view_size[0] = 2*self.w_r
    w_f_pxl = int((self.w_f/self.w_r)*img.shape[1])
    self.area_of_view_pxl_size[0] = img.shape[1]
    self.area_of_view_pxl_size[1] = int(img.shape[1]*self.area_of_view_size[1]/self.area_of_view_size[0])


    pts_src = np.array([[0,0],[img.shape[1],0],[img.shape[1],img.shape[0]],[0,img.shape[0]]])
    pts_dst = np.array([[0,0],[self.area_of_view_pxl_size[0],0],[int((self.area_of_view_pxl_size[0]+w_f_pxl)/2),self.area_of_view_pxl_size[1]],[int((self.area_of_view_pxl_size[0]-w_f_pxl)/2),self.area_of_view_pxl_size[1]]])
    h_m, status = cv2.findHomography(pts_src, pts_dst)
    rectified_img = cv2.warpPerspective(img, h_m, (self.area_of_view_pxl_size[0],self.area_of_view_pxl_size[1]))
    self.aov_object_coords = []
    if len(detections) != 0:
      for det in detections:
        x11,y11,x22,y22 = det[0],det[1],det[2],det[3]
        x_c = int((x11+x22)/2)
        y_c = int((y11+y22)/2)
        frame_coords_pxl = np.array([[x_c],[y_c],[1]])
        aov_object_coords_pxl = np.matmul(h_m,frame_coords_pxl)
        y_o = float(self.area_of_view_size[1]*(1 - aov_object_coords_pxl[1]/self.area_of_view_pxl_size[1])+self.d_f)
        x_o = float(self.area_of_view_size[0]*aov_object_coords_pxl[0]/self.area_of_view_pxl_size[0] - self.w_r)
        self.aov_object_coords.append([x_o,y_o])
    return rectified_img,self.aov_object_coords


  def draw_object_map(self,plot = False):
    # the method plots the objects map of space before robot
    object_pxl_map = np.ones((int(self.area_of_view_pxl_size[1]*(1+self.d_f/self.area_of_view_size[1])),self.area_of_view_pxl_size[0],3))*150
    x_r_coords = []
    y_r_coords = []
    for obj in self.aov_object_coords:
      x_r = obj[0]
      x_r_coords.append(x_r)
      y_r = obj[1]
      y_r_coords.append(y_r)
      x_pxl = int(self.area_of_view_pxl_size[0]*(x_r+self.w_r)/self.area_of_view_size[0])
      y_pxl = int(self.area_of_view_pxl_size[1]*(1 - (y_r-self.d_f)/self.area_of_view_size[1]))
      object_pxl_map = cv2.circle(object_pxl_map, (x_pxl,y_pxl), radius=10, color=(100, 0, 200), thickness=10)
    if plot and x_r_coords and y_r_coords:
      ax = plt.gca()
      ax.spines['top'].set_color('none')
      ax.spines['left'].set_position('zero')
      ax.spines['right'].set_color('none')
      ax.spines['bottom'].set_position('zero')

      plt.xlim(-self.w_r,self.w_r)
      plt.ylim(0,self.d_r)
      plt.scatter(x_r_coords, y_r_coords)
      plt.grid(True)
      plt.show()


    return object_pxl_map


img = cv2.imread('/content/drive/MyDrive/GarbageClassifier/TACO/test_imgs_my_in/2objects_to_trnaform.jpg')
#cv2_imshow(img)

iph4 = camera(math.radians(30.41),math.radians(23.75))
img_rectified,obj_coords = iph4.rectify_perspective(img,0.5,math.radians(45),[[679,216,900,393]])
cv2_imshow(img_rectified)

cv2_imshow(iph4.draw_object_map(plot = True))


In [ ]:
print(iph4.d_f,iph4.d_r,iph4.w_f,iph4.w_r)

0.1898923261425237 1.1632908211056585 0.2809855379810598 0.6954636110570259


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies
%cd /content/yolov5

In [ ]:
import torch
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import sys
sys.path.append('/content/yolov5')
from models.common import DetectMultiBackend
from utils.augmentations import  letterbox
from utils.general import non_max_suppression,scale_coords

class camera():
  def __init__(self,hor_fov,ver_fov,height = None,tilt_a = None):
    self.hor_fov = hor_fov
    self.ver_fov = ver_fov
    self.h = height
    self.tilt_a = tilt_a
    self.d_f =None
    self.d_r =None
    self.w_f =None
    self.w_r =None
    self.area_of_view_size = [None,None]
    self.area_of_view_pxl_size = [None,None]
    self.aov_object_coords = None # relative to robot

  def rectify_perspective(self,img,h = None,tilt_a = None,detections = []):
    if h is not None:
      self.h = h
    if tilt_a is not None:
      self.tilt_a = tilt_a
    if img is None:
      print('Given images is None')
      return
    self.d_f = self.h*tan(self.tilt_a - self.ver_fov)
    self.d_r = self.h*tan(self.tilt_a + self.ver_fov)
    self.area_of_view_size[1] = self.d_r - self.d_f

    self.w_f = sqrt(self.d_f**2+self.h**2)*tan(self.hor_fov)
    self.w_r = sqrt(self.d_r**2+self.h**2)*tan(self.hor_fov)
    self.area_of_view_size[0] = 2*self.w_r
    w_f_pxl = int((self.w_f/self.w_r)*img.shape[1])
    self.area_of_view_pxl_size[0] = img.shape[1]
    self.area_of_view_pxl_size[1] = int(img.shape[1]*self.area_of_view_size[1]/self.area_of_view_size[0])


    pts_src = np.array([[0,0],[img.shape[1],0],[img.shape[1],img.shape[0]],[0,img.shape[0]]])
    pts_dst = np.array([[0,0],[self.area_of_view_pxl_size[0],0],[int((self.area_of_view_pxl_size[0]+w_f_pxl)/2),self.area_of_view_pxl_size[1]],[int((self.area_of_view_pxl_size[0]-w_f_pxl)/2),self.area_of_view_pxl_size[1]]])
    h_m, status = cv2.findHomography(pts_src, pts_dst)
    rectified_img = cv2.warpPerspective(img, h_m, (self.area_of_view_pxl_size[0],self.area_of_view_pxl_size[1]))
    self.aov_object_coords = []
    if len(detections) != 0:
      for det in detections:
        x11,y11,x22,y22 = det[0],det[1],det[2],det[3]
        x_c = int((x11+x22)/2)
        y_c = int((y11+y22)/2)
        frame_coords_pxl = np.array([[x_c],[y_c],[1]])
        aov_object_coords_pxl = np.matmul(h_m,frame_coords_pxl)
        y_o = float(self.area_of_view_size[1]*(1 - aov_object_coords_pxl[1]/self.area_of_view_pxl_size[1])+self.d_f)
        x_o = float(self.area_of_view_size[0]*aov_object_coords_pxl[0]/self.area_of_view_pxl_size[0] - self.w_r)
        self.aov_object_coords.append([x_o,y_o])
    return rectified_img,self.aov_object_coords


  def draw_object_map(self,plot = False):
    # the method plots the objects map of space before robot
    object_pxl_map = np.ones((int(self.area_of_view_pxl_size[1]*(1+self.d_f/self.area_of_view_size[1])),self.area_of_view_pxl_size[0],3))*150
    x_r_coords = []
    y_r_coords = []

    for obj in self.aov_object_coords:
      x_r = obj[0]
      x_r_coords.append(x_r)
      y_r = obj[1]
      y_r_coords.append(y_r)
      x_pxl = int(self.area_of_view_pxl_size[0]*(x_r+self.w_r)/self.area_of_view_size[0])
      y_pxl = int(self.area_of_view_pxl_size[1]*(1 - (y_r-self.d_f)/self.area_of_view_size[1]))
      object_pxl_map = cv2.circle(object_pxl_map, (x_pxl,y_pxl), radius=10, color=(100, 0, 200), thickness=10)
    if plot and x_r_coords and y_r_coords:
      ax = plt.gca()
      ax.spines['top'].set_color('none')
      ax.spines['left'].set_position('zero')
      ax.spines['right'].set_color('none')
      ax.spines['bottom'].set_position('zero')

      plt.xlim(-self.w_r,self.w_r)
      plt.ylim(0,self.d_r)
      plt.scatter(x_r_coords, y_r_coords)
      plt.grid(True)
      plt.show()


    return object_pxl_map

def detect_sheet(model,weights,conf_thres,path2img,path2save,camera_class,device):
  device = torch.device(device)
  class_mapping = {0:'glass',1:'metal',2:'paper',3:'plastic'}
  img = cv2.imread(path2img)
  if img is not None:

    img2feed = letterbox(img.copy(), (640,640), stride=32, auto=True)[0]
    img2feed = img2feed.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img2feed = np.ascontiguousarray(img2feed)
    img2feed = torch.from_numpy(img2feed).to(device)
    img2feed = img2feed.float()
    img2feed /= 255
    img2feed = img2feed[None]

    pred = model(img2feed, augment=False, visualize=False)

    pred = non_max_suppression(pred, conf_thres = conf_thres, iou_thres = 0.5, classes = None, agnostic = False, max_det=1000)
    i = 1

    for idx in range(len(pred)):
      pred[idx][:, :4] = scale_coords(img2feed.shape[2:], pred[idx][:, :4], img.shape).round()
    #print(pred)
      img_rectified,obj_coords = camera_class.rectify_perspective(img,0.7,math.radians(20),reversed(pred[idx]))


      for *xyxy, conf, cls in reversed(pred[idx]):
        i += 1
        xyxy = [int(c.tolist()) for c in xyxy]
        t_l,b_r = (xyxy[0],xyxy[1]),(xyxy[2],xyxy[3])
        t_r = (b_r[0],t_l[1])
        b_l = (t_l[0],b_r[1])
        points = np.array([t_l,t_r,b_r,b_l])

        cv2.polylines(img,pts = [points],isClosed = True, color = (0, 200, 100), thickness = 4)
        img = cv2.putText(img, class_mapping[cls.item()-i]+':'+str(round(conf.item(),2)), t_l, cv2.FONT_HERSHEY_SIMPLEX,3, (255, 0, 0), 5, cv2.LINE_AA)


    cv2_imshow(img)
    cv2_imshow(img_rectified)
    camera_class.draw_object_map(True)


shown = 0
s_dir = '/content/drive/MyDrive/GarbageClassifier/TACO/test_imgs_my_in'

d_dir = '/content/drive/MyDrive/GarbageClassifier/TACO/test_imgs_my_out'

weights = '/content/drive/MyDrive/GarbageClassifier/TACO/yolo_taco/best_100epochs.pt'
device = 'cpu'

model = DetectMultiBackend(weights, device=device, dnn=False, data='/content/yolov5/data/coco128.yaml')

iph4 = camera(math.radians(30.41),math.radians(23.75))

for img_name in ['/content/drive/MyDrive/GarbageClassifier/TACO/test_imgs_my_in/2objects_to_trnaform_h0.5.jpg','/content/drive/MyDrive/GarbageClassifier/TACO/test_imgs_my_in/metal_glass_objects2detect.jpg']:

  detect_sheet(model,weights,conf_thres = 0.1,
               path2img = os.path.join(dir,img_name),path2save = os.path.join(d_dir,img_name),camera_class = iph4, device = device)
